In [1]:
##  Loading scraped data ..
import pandas as pd
data = pd.read_excel('scraped_data.xlsx')
data

,Date,Value,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,https://www.fool.com/investing/2019/03/16/3-to...,https://cdn.snapi.dev/images/v1/h/e/health534.jpg,3 'Top Healthcare Stocks' to Buy in March,Find out why our Motley Fool contributors call...,The Motley Fool,"Sat, 16 Mar 2019 10:36:30 -0400",Positive,Article,"CVS,HQY,PODD",\nThe healthcare market is huge. Americans alo...
1,https://www.fool.com/investing/2019/03/16/how-...,https://cdn.snapi.dev/images/v1/a/m/amc23.jpg,How Safe Are AMC Entertainment and Its Dividend?,Do the theater company's financials support it...,The Motley Fool,"Sat, 16 Mar 2019 10:31:32 -0400",Positive,Article,"AMC,NCMI",\nWhile 2018 was a record year for the domesti...
2,https://www.fool.com/investing/2019/03/16/ebay...,https://cdn.snapi.dev/images/v1/m/e/mercado56.jpg,eBay's $2.5 Billion Boondoggle,In the wake of PayPal's $750 million investmen...,The Motley Fool,"Sat, 16 Mar 2019 10:28:37 -0400",Positive,Article,"EBAY,EBAYL,MELI",\nLatin American e-commerce leader MercadoLibr...
3,https://www.fool.com/investing/2019/03/16/buy-...,https://cdn.snapi.dev/images/v1/u/r/urlhttps3a...,"Buy OrganiGram Holdings, Not Cronos Group",Popular pot stocks don't always make for the b...,The Motley Fool,"Sat, 16 Mar 2019 10:25:59 -0400",Neutral,Article,"CRON,OGRMF",\nThere's little doubt that the fast-paced mar...
4,https://www.fool.com/investing/2019/03/16/tesl...,https://cdn.snapi.dev/images/v1/m/o/modely67.jpg,Tesla Unveils Model Y: What You Need to Know,"Late Thursday night, electric-car company Tesl...",The Motley Fool,"Sat, 16 Mar 2019 10:24:33 -0400",Positive,Article,TSLA,"\nLate Thursday night, electric-car company Te..."
...,...,...,...,...,...,...,...,...,...,...
5330,https://www.youtube.com/watch?v=WvhtYkgGdwA,https://cdn.snapi.dev/images/v1/m/q/rio-tinto-...,Rio Tinto CEO Not Worried About China,Mar.25 -- Rio Tinto CEO Jean-Sebastien Jacques...,Bloomberg Markets and Finance,"Mon, 25 Mar 2019 04:55:11 -0400",Positive,Video,RIO,NaN
5331,https://seekingalpha.com/article/4250737-imper...,https://cdn.snapi.dev/images/v1/i/m/imperial1.jpg,Imperial Oil 2019 NAV,"Between the Syncrude, Aspen and Kearl projects...",Seeking Alpha,"Mon, 25 Mar 2019 04:53:25 -0400",Positive,Article,IMO,Seeking Alpha - Go to HomepageTrendingMy Portf...
5332,https://seekingalpha.com/article/4250735-shu-p...,https://cdn.snapi.dev/images/v1/s/m/smith.jpg,SHU Portfolio; Smith Micro's Share Price Revives,We're waiting for new impulses like new carrie...,Seeking Alpha,"Mon, 25 Mar 2019 04:05:43 -0400",Positive,Article,SMSI,Seeking Alpha - Go to HomepageTrendingMy Portf...
5333,https://seekingalpha.com/article/4250732-tlt-m...,https://cdn.snapi.dev/images/v1/l/k/lko.jpg,TLT And MUB: The Only 2 Bond ETFs You Need To ...,"How to properly play the bond market, with opt...",Seeking Alpha,"Mon, 25 Mar 2019 02:37:04 -0400",Positive,Article,"MUB,TLT",Seeking Alpha - Go to HomepageTrendingMy Portf...


In [2]:
data.columns = ['news_index', 'image_url', 'title', 'text', 'source_name', 'date', 'topics', 'sentiment', 'type', 'description']
data['id'] = [i for i in range(data.shape[0])]
data.columns

Index(['news_index', 'image_url', 'title', 'text', 'source_name', 'date',
       'topics', 'sentiment', 'type', 'description', 'id'],
      dtype='object')

In [5]:
##  Setting up model and PineconeDB ..
import pinecone
from tqdm.autonotebook import tqdm
pinecone.init(api_key='21ade66f-7986-4973-8684-0f330a985509', environment='us-west1-gcp')

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2',device='cuda')
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)


In [6]:
pinecone.create_index(name='document-search', dimension=768)
index = pinecone.Index('document-search')

In [7]:
document_list = []
for i, row in data.iterrows():
    document_list.append((str(row['id']),
                        model.encode(str(row['description'])).tolist(),
                        {'title': row['title'], 'text': row['text'], 'description': row['description'],
                         'date': row['date'], 'topics': row['topics'], 'sentiment': row['sentiment'],
                         'type': row['type'], 'source_name': row['source_name'] }))

    if len(document_list)==50:
        index.upsert(vectors=document_list)
        document_list = []  

PineconeProtocolError: Failed to connect; did you specify the correct index name?

In [ ]:
query = "How do I prepare for civil service?"
xq = model.encode([query]).tolist()

result = index.query(xq, top_k=2, includeMetadata=True)
result

In [ ]:

query = "How to levergae internet for business"
xq = model.encode([query]).tolist()
result = index.query(xq, top_k=2, includeMetadata=True)
result